In [ ]:
"""

Plot melt resulting from the different percentiles of parameters

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
home_path = '/bettik/burgardc/'
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
ref_Gt_list = []
ref_box1_list = []
ds_melt_param_list = []
file_isf_list = []
ds_melt_pctl_nrun_list = []

run_list = ['OPM006','OPM016','OPM018','OPM021']
for n,nemo_run in enumerate(run_list):
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt.assign_coords({'nemo_run': nemo_run}))
        
    # File mask
    inputpath_mask = home_path+'DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)
    
    ## Melt percentiles
    outputpath_melt = home_path+'DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_BT_RESULTS/nemo_5km_'+nemo_run+'/'
    

    ds_melt_pctl_param_list = []
    for pctl in [5,33,50,66,95]:
        melt_pctl_files = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_'+str(pctl).zfill(2)+'.nc')))

        param_pctl_list = []

        for mfilename in melt_pctl_files:
            #paramname = mfilename[97:157].split('.')[0].split('_')[:-1]
            paramname = mfilename[104:157].split('.')[0].split('_')[:-2]
            #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
            mparam = '_'.join(paramname)+'_tuned'
            param_pctl_list.append(mparam)

        ds_melt_pctl_param  = xr.open_mfdataset(melt_pctl_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
        ds_melt_pctl_param = ds_melt_pctl_param.rename({'new_param': 'param'})
        ds_melt_pctl_param = ds_melt_pctl_param.assign_coords(param=param_pctl_list)
        ds_melt_pctl_param_Gt = ds_melt_pctl_param['melt_1D_Gt_per_y']
        ds_melt_pctl_param_list.append(ds_melt_pctl_param_Gt.assign_coords({'pctl': pctl}))
    
    ds_melt_pctl_nrun = xr.concat(ds_melt_pctl_param_list, dim='pctl')
    ds_melt_pctl_nrun_list.append(ds_melt_pctl_nrun.assign_coords({'nemo_run': nemo_run}))
    
file_isf_all = xr.concat(file_isf_list, dim='nemo_run').assign_coords({'nemo_run': run_list})
ds_melt_param_pctl_all = xr.concat(ds_melt_pctl_nrun_list, dim='nemo_run').assign_coords({'nemo_run': run_list})
ref_Gt_all = xr.concat(ref_Gt_list, dim='nemo_run').assign_coords({'nemo_run': run_list})


In [ ]:
ds_melt_param_pctl_all.sel(nemo_run='OPM006', profile_domain=50)

In [ ]:
nemo_run = 'OPM021'

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ref_Gt_all.time, ref_Gt_all.sel(nemo_run=nemo_run, Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_pctl_all.param):
    #for n,nrun in enumerate(['ctrl94','isf94','isfru94']):
        
        for pctl in ds_melt_param_pctl_all.pctl:
            ax[i].plot(ref_Gt_all.time, ds_melt_param_pctl_all.sel(nemo_run='OPM006', 
                                                                   profile_domain=50,
                                                                   Nisf=kisf,
                                                                   pctl=pctl,
                                                                   param=mparam), color=colors[n],alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_pctl_timeseries_'+nemo_run+'.pdf')

In [ ]:
ds_melt_param_pctl_all.param

In [ ]:
ds_melt_param_all

In [ ]:
ds_melt_param_all.sel(param='boxes_4_pismyes_picopno_tuned')